In [179]:
import os
import pickle
import random
from collections import defaultdict
import numpy as np
import scipy.io
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

In [180]:
twenty = fetch_20newsgroups(subset='all', shuffle=False, remove=('headers', 'footers'))

In [181]:
print('Number of articles: ' + str(len(twenty.data)))
print('Number of different categories: ' + str(len(twenty.target_names)))
twenty.target_names

Number of articles: 18846
Number of different categories: 20


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [182]:
select_targets = set()
rare_targets = set()
select_target_names = set(twenty.target_names)
rare_target_names = set()
#select_target_names = {'rec.autos', 'alt.atheism', 'sci.med', 'comp.sys.ibm.pc.hardware'}
rare_target_names = {'rec.autos', 'rec.motorcycles'}

for i, name in enumerate(twenty.target_names):
    if name in rare_target_names:
        rare_targets.add(i)
    if name in select_target_names:
        select_targets.add(i)

In [183]:
twenty_grouped = defaultdict(list)

for i, article in enumerate(twenty.data):
    group_num = twenty.target[i]
    if group_num in select_targets:
        twenty_grouped[group_num].append((group_num, article))

In [184]:
for k in twenty_grouped.keys():
    if k in rare_targets:
        twenty_grouped[k] = twenty_grouped[k][: int(len(twenty_grouped[k]) * 0.2)]
    print(len(twenty_grouped[k]))

994
963
991
799
988
990
984
940
999
975
198
985
199
987
973
982
910
775
628
997


In [185]:
# Split equally by group; returns (group index, data) pair
def tr_va_ts_split(grouped, tr_prop, va_prop, ts_prop):
    assert tr_prop + va_prop + ts_prop == 1.
    train, valid, test = list(), list(), list()
    for i in range(len(grouped.keys())):
        num_tr = int(tr_prop * len(grouped[i]))
        num_va = int(va_prop * len(grouped[i]))
        train.extend(grouped[i][: num_tr])
        valid.extend(grouped[i][num_tr : (num_tr + num_va)])
        test.extend(grouped[i][(num_tr + num_va) :])
    random.Random(5).shuffle(train)
    random.Random(5).shuffle(valid)
    random.Random(5).shuffle(test)
    return train, valid, test

In [186]:
# List of weird params: train/valid split (0.1 for 7 topics, 0.05 else for va), and min_df=50 vs. min_df=10 vs. min_df=30 change for 7 topics vs. 4 topics vs. rare topic (all these same for comparisons within a type of test, so should be fine)
#train, valid, test = tr_va_ts_split(twenty_grouped, 0.60, 0.1, 0.3)
train, valid, test = tr_va_ts_split(twenty_grouped, 0.65, 0.05, 0.3)

In [187]:
print(len(train))
print(len(valid))
print(len(test))

11209
852
5196


In [188]:
tf_vect = TfidfVectorizer(stop_words=stopwords.words('english'),
                          use_idf=False,
                          norm=None,
                          token_pattern=r"(?u)\b[a-zA-Z][a-zA-Z]+\b")

# drop docs that don't have at least min_cnt words (can only check after tfidf transform)
def split_and_drop(mat, labels, min_cnt=10, drop=True, verbose=True):
    counts = np.asarray(np.split(mat.data.astype(np.uint8), mat.indptr[1:-1]))
    tokens = np.asarray(np.split(mat.indices.astype(np.uint16), mat.indptr[1:-1]))
    small_idxs = []
    if drop:
        for i in range(len(counts)):
            if counts[i].sum() < min_cnt:
                small_idxs.append(i)
        if verbose:
            print(f'Deleted {len(small_idxs)} docs with <{min_cnt} words')
    return np.delete(counts, small_idxs), np.delete(tokens, small_idxs), np.delete(labels, small_idxs), small_idxs

def split_and_drop_mult(mats, labels, min_cnt=10, verbose=True):
    counts_list, tokens_list = [], []
    small_idxs = set()
    for j, mat in enumerate(mats):
        if j > 0:
            min_cnt = 1
        counts = np.asarray(np.split(mat.data.astype(np.uint8), mat.indptr[1:-1]))
        tokens = np.asarray(np.split(mat.indices.astype(np.uint16), mat.indptr[1:-1]))
        counts_list.append(counts)
        tokens_list.append(tokens)
        for i in range(len(counts)):
            if counts[i].sum() < min_cnt:
                small_idxs.add(i)
    if verbose:
        print(f'Deleted {len(small_idxs)} docs with <{min_cnt} words')
    small_idxs = list(small_idxs)
    for i in range(len(mats)):
        counts_list[i] = np.delete(counts_list[i], small_idxs)
        tokens_list[i] = np.delete(tokens_list[i], small_idxs)
    labels = np.delete(labels, small_idxs)
    return counts_list, tokens_list, labels, small_idxs

def process(train, valid, test):
    tr_labels, tr_data = [list(t) for t in zip(*train)]
    va_labels, va_data = [list(t) for t in zip(*valid)]
    ts_labels, ts_data = [list(t) for t in zip(*test)]
    
    tf_vect.set_params(min_df=30, max_df=0.7, vocabulary=None)
    tr_mat = tf_vect.fit_transform(tr_data).sorted_indices()
    vocab = tf_vect.get_feature_names()
    
    tf_vect.set_params(min_df=1, max_df=1., vocabulary=vocab)
    vocab2 = tf_vect.get_feature_names()
    va_mat = tf_vect.fit_transform(va_data).sorted_indices()
    ts_mat = tf_vect.fit_transform(ts_data).sorted_indices()
    
    tr_counts, tr_tokens, tr_labels, _ = split_and_drop(tr_mat, tr_labels)
    va_counts, va_tokens, va_labels, _ = split_and_drop(va_mat, va_labels)
    
    ts_clean_data = ts_data
    ts_h1_data = [article[: len(article) // 2] for article in ts_clean_data]
    ts_h2_data = [article[len(article) // 2 :] for article in ts_clean_data]
    ts_h1_mat = tf_vect.fit_transform(ts_h1_data).sorted_indices()
    ts_h2_mat = tf_vect.fit_transform(ts_h2_data).sorted_indices()
    ts_counts, ts_tokens, ts_labels, _ = split_and_drop_mult([ts_mat, ts_h1_mat, ts_h2_mat], ts_labels)
    counts = [tr_counts, va_counts] + ts_counts
    tokens = [tr_tokens, va_tokens] + ts_tokens
    return counts, tokens, [tr_labels, va_labels, ts_labels], vocab
    
def save(counts, tokens, labels, vocab, path, prefix):
    with open(os.path.join(path, 'vocab.pkl'), 'wb') as f:
        pickle.dump(vocab, f)
    with open(os.path.join(path, 'labels.pkl'), 'wb') as f:
        pickle.dump({'train': labels[0], 'valid': labels[1], 'test': labels[2]}, f)
    for i, name in enumerate(['tr', 'va', 'ts', 'ts_h1', 'ts_h2']):
        scipy.io.savemat(os.path.join(path, f'{prefix}_{name}_counts.mat'), {'counts': counts[i]})
        scipy.io.savemat(os.path.join(path, f'{prefix}_{name}_tokens.mat'), {'tokens': tokens[i]})
    print('Saved!')

In [189]:
data_path = './../data/my_20ng_rare'
counts, tokens, labels, vocab = process(train, valid, test)
if not os.path.exists(data_path):
    os.mkdir(data_path)
save(counts, tokens, labels, vocab, data_path, 'bow')

Deleted 465 docs with <10 words
Deleted 36 docs with <10 words
Deleted 245 docs with <1 words
Saved!


In [190]:
print(f'Num train articles: {len(counts[0])}')
print(f'Num valid articles: {len(counts[1])}')
print(f'Num test articles:  {len(counts[2])}')
print(f'Vocab size: {len(vocab)}')

Num train articles: 10744
Num valid articles: 816
Num test articles:  4951
Vocab size: 5350
